<a href="https://colab.research.google.com/github/abaranguer/tfm-muecim/blob/main/ColanNotebooks/start_colab_tfm_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# START TFM WEB FRONTEND

https://deveshsurve.medium.com/running-flask-app-with-google-colab-c9f0c078fb53


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/TFM-MUECIM/tfm_www.tar .
!cd /content; tar xf tfm_www.tar
!cp -r /content/drive/MyDrive/TFM-MUECIM/src /content/src
!cp -r /content/drive/MyDrive/TFM-MUECIM/sh /content/sh
!cp -r /content/drive/MyDrive/TFM-MUECIM/ipynb /content/ipynb
!cp -r /content/drive/MyDrive/TFM-MUECIM/txt /content/txt
!cp -r /content/drive/MyDrive/TFM-MUECIM/trained-models /content/trained-models
!rm /content/tfm_www.tar
!cp sh/tar_tfm_www.sh .
!cp sh/copy_tfm_www.sh .

cp: cannot stat 'sh/copy_tfm_www.sh': No such file or directory


In [3]:
from google.colab.output import eval_js
print(f'Per a executar el frontend web del TFM, fes click a {eval_js("google.colab.kernel.proxyPort(5000)")}/index.html')

Per a executar el frontend web del TFM, fes click a https://5000-m-s-3ris64yzlouiw-a.asia-east1-0.prod.colab.dev/index.html


In [4]:
import sys
baseDir = '/content' #/drive/My Drive/TFM-MUECIM'
sys.path.append(baseDir)

In [6]:
from tfm_EURLEX57KDataset import EURLEX57KDataset

ds = EURLEX57KDataset(baseDir,'tfm_www/csv/TokenizedBERTDataFrame.csv')

In [7]:
from tfm_50LabelsLabelIndex import LabelIndex

datFolder = '/content/tfm_www/dat'
labelIndex = LabelIndex(datFolder)

In [8]:
import os
import torch
from transformers import AutoTokenizer, BertModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'

modelFolder = '/content/tfm_www/models'
modelFile = '20250216_50L_tfm_BERT_model_epoch_6.pt'

torch.serialization.add_safe_globals([BertModel])

model = torch.load(os.path.join(modelFolder, modelFile),
                   map_location=torch.device(device),
                   weights_only=False)


In [9]:
item = ds.__getitem__(29065)

outputs = model(
   input_ids=item['input_ids'][0:512].unsqueeze(0),
   attention_mask=item['attention_mask'][0:512].unsqueeze(0),
   labels=item['labels'].unsqueeze(0))

In [10]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
# calculate metrics
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0])
threshold = 0.5
y_pred = np.zeros(probs.shape)
y_pred[np.where(probs >= threshold)] = 1
y_true = item['labels'].cpu().numpy() # Convert y_true to a NumPy array
f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
accuracy = accuracy_score(y_true, y_pred)

metrics = {'f1': f1_micro_average,
          'roc_auc': roc_auc,
          'accuracy': accuracy}



In [11]:
metrics

{'f1': 1.0, 'roc_auc': 1.0, 'accuracy': 1.0}

In [12]:
i = 0
for y in zip(y_true, y_pred):
  if y[0] == 1 and y[1] == 1:
    print(f'Correcte: {labelIndex.id2label[i]}')
  if y[0] != y[1]:
    if y[0] == 1:
      print(f'Error: ground truth: {labelIndex.id2label[i]}, però no predit.')
    else:
      print(f'Error: predit: {labelIndex.id2label[i]}, però no al ground truth.')
  i += 1

Correcte: 4743
